In [2]:
import glob

# --- 1. Define Constants ---
# These must match what you used during export!
BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'B12']
LABEL_BAND = 'label'
ALL_BANDS = BANDS + [LABEL_BAND]
PATCH_SIZE = 256
NUM_CLASSES = 9 # Number of Dynamic World classes
BATCH_SIZE = 32 # Adjust based on your GPU memory
BUFFER_SIZE = 2000 # For shuffling
DATA_DIR = 'Data\tfrecords'

In [3]:

# --- 2. Find Your TFRecord Files ---
# Use glob to create a list of all your exported files.
tfrecord_files = glob.glob(DATA_DIR + 'training_patches*.tfrecord.gz')
print(f"Found {len(tfrecord_files)} TFRecord files.")

# --- 3. Create a TFRecord Dataset Object ---
# This object knows how to read the files efficiently.
dataset = tf.data.TFRecordDataset(tfrecord_files, compression_type='GZ')

Found 0 TFRecord files.


NameError: name 'tf' is not defined

In [ ]:


# --- 4. Define a Parsing Function ---
# This function tells TensorFlow how to interpret the raw bytes in the TFRecord.
def parse_tfrecord(example_proto):
    # Define the structure of the data.
    feature_description = {
        band: tf.io.FixedLenFeature([PATCH_SIZE, PATCH_SIZE], tf.float32) for band in BANDS
    }
    feature_description[LABEL_BAND] = tf.io.FixedLenFeature([PATCH_SIZE, PATCH_SIZE], tf.int64)

    # Parse the input `tf.train.Example` proto using the dictionary above.
    example = tf.io.parse_single_example(example_proto, feature_description)
    
    # Stack the feature bands into a single tensor (the image).
    features = [example[band] for band in BANDS]
    image_stack = tf.stack(features, axis=-1)
    
    # Get the label band.
    label = example[LABEL_BAND]
    
    # The label needs to be one-hot encoded for segmentation tasks.
    label_one_hot = tf.one_hot(label, depth=NUM_CLASSES)
    
    return image_stack, label_one_hot